In [ ]:
# azure openai from langchain

In [16]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os




In [ ]:
load_dotenv()

# Variables necesarias para Azure OpenAI
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("03_MINI_DEPLOYMENT")  # nombre del *deployment*, NO del modelo
api_version = os.getenv("AZURE_OPENAI_API_VERSION")  # Ajusta según la versión de tu Azure OpenAI


# ------------------ 3. Configurar modelo Azure ------------------ #
llm = AzureChatOpenAI(
    openai_api_key=api_key,
    azure_endpoint=api_base,
    deployment_name=deployment,
    api_version=api_version,
    # temperature=0
)

In [15]:
response = llm.invoke("hola")
print(response.content)

¡Hola! ¿En qué puedo ayudarte hoy?


In [53]:
# test lectura de ficha

from langchain.document_loaders import TextLoader
from langchain_core.runnables import Runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import SystemMessage

In [126]:
# load files
aid_name = "actuación_conjunta_isciii-cdti"
# aid_name = "proyectos_de_i_+_d"
# aid_name = "proyectos_de_i+d_aeroespacial_y_salud"

aid_path = f"data/cdti/cdti-aids/{aid_name}"
description_path = f"{aid_path}/{aid_name}_description.md"
card_path = f"{aid_path}/{aid_name}_card.md"


# description
loader = TextLoader(description_path, encoding="utf-8")
docs = loader.load()
description = docs[0].page_content  # asumimos que es un único documento

# card
loader = TextLoader(card_path, encoding="utf-8")
docs = loader.load()
card = docs[0].page_content  # asumimos que es un único documento

# join
document = description + "\n\n" + card


In [152]:
EXTRACTION_SYSTEM_MESSAGE = "Eres un asistente que extrae información de fichas administrativas en español."

EXTRACTION_HUMAN_MESSAGE = """
A continuación tienes el contenido de una ficha técnica de una subvención en formato Markdown.

Devuelve un objeto JSON con las siguientes claves extraídas del texto:
- "organismo": Entidad que da la subvención. Algunos ejemplos son CDTI, SODERCAN, SPRI, etc.
- "nombre": Título de la ayuda o subvención. Suele ser la primera línea del texto
- "linea": Modalidades de la subvención. Solo la ayuda llamada "proyectos de I+D" tiene 5 lineas de ayudas llamadas modalidades. Devolver una lista de python esas 5 modalidades. Esl resto devolver una lista vacía ("[]").
- "fecha_inicio": Fecha de inicio del plazo para presentar la solicitud, en formato dd/mm/yyyy si es posible. Si es todo el año, devolver el string "Todo el año" en fecha inicio y fecha fin.
- "fecha_fin": Fecha de fin del plazo para presentar la solicitud, en formato dd/mm/yyyy si es posible. Si es todo el año, devolver el string "Todo el año" en fecha inicio y fecha fin.
- "objetivo: Objetivo general de la ayuda o actuación. Si existe el parrafo "Objetivo General de la actuación", devolver ese texto. En caso contrario, se construye un breve párrafo de objetivo de la ayuda a partir del texto Usa verbos en formato sustantivo (Creación en vez de crear, apoyo en vez de apoyar, etc).
- "beneficiarios": Beneficiarios de la ayuda o actuación. Si existe el parrafo "Beneficiarios", devolver ese texto. En caso contrario, se construye una frase breve de beneficiarios a partir del texto.
No expliques nada. Solo responde con un JSON válido.
- "anio": Año de la convocatoria. Si no hace referencia al año, devolver el año actual (2025).
- "area": Clasifica la convocatoria en una de las siguientes areas: [I+D, Innovación, Inversión, Internacional]
- "presupuesto_minimo": Extrae el valor del presupuesto mínimo. Si no existe, devolver "".
- "duración_mínima": Extrae la duración mínima de la ayuda en formato string ("<duracion> meses"). Si no existe, devolver "".
- "duración_máxima": Extrae la duración máxima de la ayuda en formato string ("<duracion> meses"). Si aparece una fecha, calcula la duración máxima en meses desde la fecha de inicio de la convocatoria. Si no existe, devolver "".
Contenido:
--------------------
{document}
--------------------
"""

In [153]:
# 4. Prompt que pide varios campos
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=EXTRACTION_SYSTEM_MESSAGE),
    ("human", EXTRACTION_HUMAN_MESSAGE)
])

# 5. Parser JSON para extraer la salida limpia
json_parser = JsonOutputParser()

# 6. Encadenar todo
chain = prompt | llm | json_parser



In [154]:
# 7. Ejecutar cadena con el documento
resultado = chain.invoke({"document": document})

In [155]:
resultado


{'organismo': 'CDTI',
 'nombre': 'Actuación conjunta ISCIII-CDTI',
 'linea': [],
 'fecha_inicio': '10/07/2023',
 'fecha_fin': '28/07/2023',
 'objetivo': 'Impulso del empleo de calidad, mejora de la salud de la población, coordinación de capacidades científicas y empresariales, y fomento del desarrollo de productos de diagnóstico clínico y medicamentos estratégicos en el marco del PERTE de Salud.',
 'beneficiarios': 'Empresas.',
 'anio': 2023,
 'area': 'I+D',
 'presupuesto_minimo': '175.000 euros',
 'duración_mínima': '12 meses',
 'duración_máxima': '30 meses'}

In [102]:
type(resultado)

dict

In [103]:
type(resultado['linea'])

list